In [ ]:
# Package to Connect to Drive
from google.colab import drive
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Mount to Google Drive
drive.mount('/content/drive')

# Move to Location, you can change the directory below to move to any location

Mounted at /content/drive


In [ ]:
%cd /content/drive/Shareddrives/SOSCD-paraphrase/

/content/drive/Shareddrives/SOSCD-paraphrase


In [ ]:
import xml.etree.ElementTree as ET

import pandas as pd

def read_info(file_name):
	xml_data = open(file_name,'r').read()
	root = ET.XML(xml_data)
	data = []
	for child in root:
		data.append([subchild.text] for subchild in child)
	df = pd.DataFrame(data)d
	return df

In [ ]:
!ls Corpus/Corpus

ETPC_SQL_v0.95.txt  paraphrase_types.xml  textual_np_pos.xml
ETPC_XML_v0.95.txt  text_pairs.xml	  textual_paraphrases.xml
negation.xml	    textual_np_neg.xml


In [ ]:
paraphrase_types = read_info('./Corpus/Corpus/paraphrase_types.xml')

In [ ]:
textual_paraphrases_pos = read_info('./Corpus/Corpus/textual_np_pos.xml')

In [ ]:
df = pd.concat([textual_paraphrases_pos[2].apply(lambda x: x[0]), textual_paraphrases_pos[6].apply(lambda x: x[0]), textual_paraphrases_pos[7].apply(lambda x: x[0])], axis = 1)

df = df[~((df[6].isnull()) | (df[7].isnull()))]

In [ ]:
df.columns = ['type', 'sentence1', 'sentence2']

In [ ]:
df['sentence1'] = df['sentence1'].str.lower()
df['sentence2'] = df['sentence2'].str.lower()
df['type'] = df['type'].str.lower()

In [ ]:
df['pairs'] = df['sentence1'] + ' [SEP] ' + df['sentence2']

In [ ]:
df_word_count = df['pairs']
sum = 0 
len_list = []
for i in range(len(df_word_count)):
  sum += len(df_word_count.iloc[i].split())
  len_list.append(len(df_word_count.iloc[i].split()))
print(sum)
print(max(len_list))
print(min(len_list))
print(np.std(len_list))

56783
66
3
11.500764320655195


In [ ]:
sumlist = [] 
for i in range(len(df_word_count)):
  sumlist += df_word_count.iloc[i].split()
print(len(set(sumlist)))

6139


In [ ]:
annotations_file = open('./annotation.txt','r')
df_new = df
for eachline in annotations_file:
  eachline = eachline.lower()
  eachline_split = eachline.split('\t')
  df_new = df_new.append({'type':eachline_split[2][:-1],'sentence1':eachline_split[0],'sentence2':eachline_split[1],'pairs':eachline_split[0]+' [SEP] '+eachline_split[1]}, ignore_index=True)

In [ ]:
df['type'].unique()

array(['identity', 'inflectional changes',
       'same polarity substitution (contextual)',
       'same polarity substitution (habitual)',
       'synthetic/analytic substitution', 'change of format',
       'change of order', 'subordination and nesting changes',
       'same polarity substitution (named ent.)', 'punctuation changes',
       'spelling changes', 'syntax/discourse structure changes',
       'ellipsis', 'modal verb changes', 'diathesis alternation',
       'converse substitution', 'derivational changes', 'entailment',
       'coordination changes', 'direct/indirect style alternations',
       'addition/deletion', 'sentence modality changes', 'non-paraphrase',
       'negation switching', 'opposite polarity substitution (habitual)',
       'opposite polarity substitution (contextual)'], dtype=object)

In [ ]:
update = df['type'].isin(["opposite polarity substitution (contextual)","opposite polarity substitution (habitual)"])
df.loc[update,'type'] = 'opposite polarity substitution'
update = df['type'].isin(["same polarity substitution (contextual)","same polarity substitution (habitual)", "same polarity substitution (named ent.)","same polarity substitution (named entities)"])
df.loc[update,'type'] = 'same polarity substitution'
update = df['type'].isin(["modal verb changes"])
df.loc[update,'type'] = 'functional word substitution'
update = df['type'].isin(["semantic based"])
df.loc[update,'type'] = 'relational substitution'

In [ ]:
print(df['type'].unique())
print(len(df))


['identity' 'inflectional changes' 'same polarity substitution'
 'synthetic/analytic substitution' 'change of format' 'change of order'
 'subordination and nesting changes' 'punctuation changes'
 'spelling changes' 'syntax/discourse structure changes' 'ellipsis'
 'functional word substitution' 'diathesis alternation'
 'converse substitution' 'derivational changes' 'entailment'
 'coordination changes' 'direct/indirect style alternations'
 'addition/deletion' 'sentence modality changes' 'non-paraphrase'
 'negation switching' 'opposite polarity substitution']
4422


In [ ]:
df_new_word_count = df_new['pairs']
sum = 0 
len_list = []
for i in range(len(df_new_word_count)):
  Ltem = len(df_new_word_count.iloc[i].split())
  sum += Ltem
  len_list.append(Ltem)
print(sum)

sumlist = [] 
for i in range(len(df_new_word_count)):
  sumlist += df_new_word_count.iloc[i].split()
print(len(set(sumlist)))

print(max(len_list))
print(min(len_list))
print(np.std(len_list))

96865
12205
279
3
13.815996118850903


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 15.6 MB/s 
     |████████████████████████████████| 163 kB 58.6 MB/s 
     |████████████████████████████████| 7.6 MB 54.3 MB/s 


In [ ]:
import torch
from sklearn.metrics import f1_score
# Creating Dataloader
from torch.utils.data import DataLoader, RandomSampler
from torch.utils.data import TensorDataset
from tqdm.notebook import tqdm
from transformers import AutoTokenizer, BertForSequenceClassification

# from utils import f1_score_func, evaluate

# The Function to Create both Word-level and Character-level Features for BERT
def get_name_pair(s):
    return s, ' '.join(str(s)).replace('  ', ' ').replace('  ', ' ')

In [ ]:
# Check All Classes
possible_labels = df['type'].unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [ ]:
df['label'] = df['type'].replace(label_dict)

In [ ]:
torch.set_grad_enabled(True)

# Store the model we want to use
MODEL_NAME = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Tokenize Data
encoded_train = tokenizer.batch_encode_plus(
    df['pairs'].values.tolist(),
    return_attention_mask=True,
    padding=True,
    return_tensors='pt')


input_ids_train = encoded_train['input_ids']
attention_masks_train = encoded_train['attention_mask']
labels_train = torch.tensor(df.label.values)

# Create datasets
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# New Section

In [ ]:
del model

In [ ]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Gen RAM Free: 9.8 GB  |     Proc size: 4.8 GB
GPU RAM Free: 9197MB | Used: 5912MB | Util  39% | Total     15109MB


In [ ]:
from sklearn.model_selection import KFold
batch_size = 16
k_folds = 5
# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)

dataloader_train = DataLoader(dataset_train,
                              #sampler=RandomSampler(dataset_train),
                              batch_size=batch_size)


In [ ]:
from transformers import AdamW

# Optimizer
optimizer = AdamW(model.parameters(),
                  lr=1e-4,
                  eps=1e-8)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
epochs = 20

In [ ]:
import numpy as np
import torch
from sklearn.metrics import f1_score


def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

# Function to Evaluate Model Performance
def evaluate(model, dataloader_val):
    model.eval()

    loss_val_total = 0
    predictions, true_vals = [], []

    for batch in dataloader_val:
        batch = tuple(b.cuda() for b in batch)

        inputs = {'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'labels': batch[2],
                  }

        with torch.no_grad():
            outputs = model(**inputs)

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)

    loss_val_avg = loss_val_total / len(dataloader_val)

    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)

    return loss_val_avg, predictions, true_vals

In [ ]:
# Training Model
for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset_train)):
  # Print
  print(f'FOLD {fold}')
  print('--------------------------------')

  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
  
  trainloader = torch.utils.data.DataLoader(
                      dataset_train, 
                      batch_size=16, sampler=train_subsampler)
  testloader = torch.utils.data.DataLoader(
                      dataset_train,
                      batch_size=16, sampler=test_subsampler)

  f1_fold = []
  f1_score_fold = []
  for epoch in tqdm(range(1, epochs + 1)):
      model.cuda()
      model.train()

      loss_train_total = 0

      progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False,
                          disable=False)
      for batch in progress_bar:
          model.zero_grad()

          batch = tuple(b.cuda() for b in batch)

          inputs = {'input_ids': batch[0],
                    'attention_mask': batch[1],
                    'labels': batch[2],
                    }

          outputs = model(**inputs)

          loss = outputs[0]
          loss_train_total += loss.item()
          loss.backward()

          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

          optimizer.step()
          progress_bar.set_postfix(
              {'training_loss': '{:.3f}'.format(loss.item() / len(batch))})

      # torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')

      tqdm.write(f'\nEpoch {epoch}')

      loss_train_avg = loss_train_total / len(trainloader)
      tqdm.write(f'Training loss: {loss_train_avg}')

      val_loss, predictions, true_vals = evaluate(model, testloader)
      val_f1 = f1_score_func(predictions, true_vals, )
      f1_fold.append(val_f1)
      tqdm.write(f'Validation loss: {val_loss}')
      tqdm.write(f'F1 Score (Weighted): {val_f1}')

  # Model Performance
  val_loss, predictions, true_vals = evaluate(model, testloader)
  f1_score_fold.append(f1_score(true_vals, np.argmax(predictions, axis=1), average=None))
f1_fold_average = torch.mean(torch.Tensor(f1_fold))
f1_score_fold_average = torch.mean(torch.Tensor(f1_score_fold))

In [ ]:
f1_score(true_vals, np.argmax(predictions, axis=1), average=None)

array([0.99478261, 0.91428571, 0.96395194, 0.95412844, 0.92857143,
       0.79245283, 0.75862069, 1.        , 0.7826087 , 0.8       ,
       0.88888889, 0.92307692, 1.        , 0.47619048, 0.85714286,
       0.8       , 0.8       , 0.        , 0.        ])

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(true_vals, np.argmax(predictions, axis=1)))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       286
           1       0.97      0.86      0.91        37
           2       0.96      0.97      0.96       373
           3       0.91      1.00      0.95        52
           4       0.87      1.00      0.93        13
           5       0.91      0.70      0.79        30
           6       0.92      0.65      0.76        17
           7       1.00      1.00      1.00        18
           8       1.00      0.64      0.78        14
           9       0.67      1.00      0.80         4
          10       0.89      0.89      0.89         9
          11       0.86      1.00      0.92         6
          12       1.00      1.00      1.00         2
          13       0.31      1.00      0.48         5
          14       1.00      0.75      0.86         4
          16       0.86      0.75      0.80         8
          17       1.00      0.67      0.80         3
          20       0.00    

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
label_dict

{'identity': 0,
 'inflectional changes': 1,
 'same polarity substitution': 2,
 'synthetic/analytic substitution': 3,
 'change of format': 4,
 'change of order': 5,
 'subordination and nesting changes': 6,
 'punctuation changes': 7,
 'spelling changes': 8,
 'syntax/discourse structure changes': 9,
 'ellipsis': 10,
 'functional word substitution': 11,
 'diathesis alternation': 12,
 'converse substitution': 13,
 'derivational changes': 14,
 'entailment': 15,
 'coordination changes': 16,
 'direct/indirect style alternations': 17,
 'addition/deletion': 18,
 'sentence modality changes': 19,
 'non-paraphrase': 20,
 'negation switching': 21,
 'opposite polarity substitution': 22}

In [ ]:
df[['type']].value_counts()

type                              
same polarity substitution            1711
identity                              1503
synthetic/analytic substitution        263
inflectional changes                   168
change of order                        152
punctuation changes                    136
subordination and nesting changes       97
spelling changes                        67
syntax/discourse structure changes      64
change of format                        62
functional word substitution            37
ellipsis                                29
derivational changes                    28
diathesis alternation                   27
coordination changes                    20
converse substitution                   16
direct/indirect style alternations      16
addition/deletion                        7
sentence modality changes                5
non-paraphrase                           5
entailment                               4
opposite polarity substitution           3
negation switching 

In [ ]:
# torch.save(model.state_dict(), './BERT_paraphrase_5epochs.pt')
torch.save(model.state_dict(), './BERT_paraphrase_10epochs.pt')

In [ ]:
%cd /content/drive/Shareddrives/SOSCD-paraphrase/

/content/drive/Shareddrives/SOSCD-paraphrase


In [ ]:
%ls 

 annotation.txt                Corpus/        sen2.txt
 BERT.ipynb                    label          totalsamples
 BERT_paraphrase_10epochs.pt   meeting.gdoc  'training data combine.ipynb'
 BERT_paraphrase_20epochs.pt   samples/
 BERT_paraphrase_5epochs.pt    sen1.txt


In [ ]:
import pandas as pd
df = pd.read_csv("./totalsamples",sep="\t|\n",header=None,names = ["id","sentence1","sentence2"])
df['sentence1'] = df['sentence1'].astype(str)
df['sentence2'] = df['sentence2'].astype(str)
df['sentence1'] = df['sentence1'].str.lower()
df['sentence2'] = df['sentence2'].str.lower()
df['pairs'] = df['sentence1'] + ' [SEP] ' + df['sentence2']
print(len(df))

7000


In [ ]:
torch.set_grad_enabled(False)

# Store the model we want to use
MODEL_NAME = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
model.load_state_dict(torch.load('./BERT_paraphrase_10epochs.pt'))
model.eval()

In [ ]:
model

In [ ]:
df['pairs'].values.tolist()

In [ ]:
# Tokenize Data
encoded_pre = tokenizer.batch_encode_plus(
    df['pairs'].values.tolist(),
    return_attention_mask=True,
    padding=True,
    truncation = True,
    return_tensors='pt')

input_ids_pre = encoded_pre['input_ids']
attention_masks_pre = encoded_pre['attention_mask']
labels_pre = torch.tensor([0] * len(df))

# Create datasets
dataset_pred = TensorDataset(input_ids_pre, attention_masks_pre, labels_pre)


In [ ]:
preloader = torch.utils.data.DataLoader(
                    dataset_pred, 
                    batch_size=1)

progress_bar = tqdm(preloader,leave=False,
                    disable=False)
for batch in progress_bar:
    model.cuda()

    batch = tuple(b.cuda() for b in batch)

    inputs = {'input_ids': batch[0],
              'attention_mask': batch[1],
              'labels': batch[2],
              }
    outputs = model(**inputs)

  0%|          | 0/7000 [00:00<?, ?it/s]

In [ ]:
loss_val_avg, predictions, true_vals =  evaluate(model,preloader)

In [ ]:
np.argmax(predictions, axis=1)

In [ ]:
df['pred'] = [possible_labels[x] for x in np.argmax(predictions, axis=1)]

In [ ]:
st,en = 0,999
L = len(df)
for i in range(0,int(L//1000)):
  print(df[['pred']].iloc[(st + 1000 * i):(en + 1000 * i)].value_counts())

pred                              
identity                              771
punctuation changes                   126
same polarity substitution             61
sentence modality changes              11
synthetic/analytic substitution        11
syntax/discourse structure changes     10
functional word substitution            5
subordination and nesting changes       2
converse substitution                   1
ellipsis                                1
dtype: int64
pred                              
identity                              681
punctuation changes                   279
same polarity substitution             15
change of order                         5
syntax/discourse structure changes      5
converse substitution                   4
subordination and nesting changes       3
ellipsis                                2
change of format                        1
coordination changes                    1
diathesis alternation                   1
sentence modality changes          